In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_pickle("testing_final_processed.pkl")
df.head(3)

,Review,Date,Stars,Restaurant,feature_descriptors,vector_descriptors,word_level_polarity,clause_level_breakdown,clause_vectors,clause_polarity,categorisation
0,There are lots of Indian restaurant at Tanjong...,2 days ago,5 stars,Gravy_Restaurant,"[{'lots': ['Indian'], 'restaurant': [], 'area'...","[{'lots': [], 'restaurant': [], 'area': [[-0.0...","[{'lots': [0.0], 'restaurant': [], 'area': [0....",[{'lots': 'There are lots of Indian restaurant...,[{'lots': [[-0.51533 0.83186 0.22457 -0.7...,"[{'lots': 0.0, 'restaurant': 0.0, 'time': 0.0,...","[{'lots': 'price', 'restaurant': 'ambience', '..."
2,"The dal makhni was sweet, seemed like sugar wa...",a week ago,2 stars,Gravy_Restaurant,"[{'dal': [], 'makhni': ['sweet', 'seemed'], 's...","[{'dal': [], 'makhni': [[ 0.10788 0.38359 ...","[{'dal': [], 'makhni': [0.35, 0.0], 'sugar': [...","[{'dal': 'The dal makhni was sweet', 'makhni':...",[{'dal': [[-0.90375 -1.43 -0.023677 -0...,"[{'dal': 0.35, 'makhni': 0.35, 'couple': 0.0, ...","[{'dal': 'food', 'makhni': 'food', 'sugar': 'p..."
4,We are regular customers - the food is amazing...,2 weeks ago,5 stars,Gravy_Restaurant,"[{'customers': [], 'food': ['had']}]","[{'customers': [], 'food': [[ 0.63256 -0.1271...","[{'customers': [], 'food': [0.0]}]","[{'customers': 'We are regular customers', 'fo...",[{'customers': [[-0.51533 0.83186 0.22457 ...,"[{'customers': 0.0, 'food': 0.6000000000000001}]","[{'customers': 'price', 'food': 'food'}]"


In [ ]:
# pd.set_option('display.max_colwidth', None)
df.head(1)[["clause_level_breakdown", "clause_vectors"]]

,clause_level_breakdown,clause_vectors
0,[{'lots': 'There are lots of Indian restaurant...,[{'lots': [[-0.51533 0.83186 0.22457 -0.7...


In [ ]:
def get_table(df):
  retdf = pd.DataFrame()
  
  for r in range(0, len(df)):
    row = df.iloc[r, :]# Get entire row
    # print(row)
    # break
    count = 0
    for feature in row["clause_vectors"][0].keys():
      rowdict = dict()
      rowdict["restaurant"] = row["Restaurant"]
      rowdict["stars"] = row["Stars"]
      rowdict["date"] = row["Date"]
      rowdict["feat_index"] = count
      count += 1
      # print("review_index", row.iloc[0])
      rowdict["index"] = row.iloc[0]
      rowdict["feature"] = feature
      try:
        rowdict["category"] = row["categorisation"][0][feature]
      except:
        rowdict["category"] = "Non-Categorised"
      # print(feature, row["clause_level_breakdown"][0][feature], '\n')
      rowdict["raw_clause"] = row["clause_level_breakdown"][0][feature]
      # print(feature, row["clause_vectors"][0][feature], '\n')
      rowdict["clause_vector"] = [row["clause_vectors"][0][feature]]
      # print(feature, row["clause_polarity"][0][feature], '\n')
      rowdict["clause_polarity"] = row["clause_polarity"][0][feature]
      
      # print(row["clause_vectors"][0][feature])
        
      # Make df row 
      newrow = pd.DataFrame.from_dict(rowdict, orient="columns")
      retdf = retdf.append(newrow)
      # return newrow
    # break
  return retdf

sentdf = get_table(df)
sentdf.head(1)

,restaurant,stars,date,feat_index,index,feature,category,raw_clause,clause_vector,clause_polarity
0,Gravy_Restaurant,5 stars,2 days ago,0,There are lots of Indian restaurant at Tanjong...,lots,price,There are lots of Indian restaurant at Tanjong...,"[[-0.51533, 0.83186, 0.22457, -0.73865, 0.1871...",0.0


In [ ]:
sentdf.to_pickle("testing_sent_df.pkl")

In [ ]:
"# USEABLE FORMAT

def get_usable(df):
  cols = list(df.columns)
  print(cols)
  cols = [i for i in cols if i != "clause_vector"]

  useable = df.groupby(cols)\
        .apply(lambda x: pd.DataFrame(x['clause_vector'].tolist()))

  # Rename columns to fit
  cols = []
  for i in useable.columns:
    cols.append("vect_"+str(i))

  useable.columns = cols

  return useable

useable = get_usable(sentdf)

['feat_index', 'index', 'feature', 'raw_clause', 'clause_vector', 'clause_polarity']


In [ ]:
useable

vect_0  ... vect_30
feat_index index feature    raw_clause                                         clause_polarity                                                       ...        
0          0     Steaks     Steaks are good of course                          0.70            0  [-0.51533, 0.83186, 0.22457, -0.73865, 0.18718...  ...     NaN
                 burgers    Only tried burgers                                 0.00            0  [0.35383, -0.52893, -0.71475, 0.14329, 0.10245...  ...     NaN
                 couple     I have been to Hashida Singapore a couple of ti... 0.80            0  [0.15711, 0.65606, 0.0021149, -0.65144, -0.284...  ...     NaN
                 food       Great food                                         0.80            0  [-0.16486, 0.91997, 0.22737, -0.49031, -0.0018...  ...     NaN
                 hamburgers Good hamburgers                                    0.70            0  [0.0068313, -0.15109, -0.012474, -0.039002, -0...  ...     NaN
...                                                                                                                                             ...  ...     ...
25         46    hands      wash their hands regularly and                     0.00            0  [-1.0274, 0.50461, 0.12139, -1.0512, -0.42454,...  ...     NaN
26         46    hair       tie up their hair                                  0.00            0  [-0.059138, -0.18104, 0.39362, -0.27191, 0.284...  ...     NaN
27         46    menu       sparkling bottled water off the menu               0.00            0  [0.20387, 0.14953, 0.49425, 0.1807, 0.33663, 0...  ...     NaN
28         46    visit      apologise that your visit this time round was n... 0.15            0  [-0.078638, 0.054143, -0.099305, 0.6109, -0.95...  ...     NaN
29         46    round      apologise that your visit this time round was n... 0.15            0  [-0.078638, 0.054143, -0.099305, 0.6109, -0.95...  ...     NaN

[14789 rows x 31 columns]

In [ ]:
# sentdf.to_pickle("total_sentiment_data.pkl")

In [ ]:
train = pd.read_pickle("train.pkl")
senttrain = get_table(train)
# senttrain.to_pickle("sent_train.pkl")

test = pd.read_pickle("test.pkl")
senttest = get_table(test)
# senttest.to_pickle("sent_test.pkl")

validate = pd.read_pickle("validate.pkl")
sentvalidate = get_table(validate)
# sentvalidate.to_pickle("sent_validate.pkl")

# ADABOOST


In [ ]:
pdf = pd.read_pickle("final_processed.pkl")
pdf.head(1)

,Unnamed: 0,Review,Date,Stars,Restaurant,feature_descriptors,vector_descriptors,word_level_polarity,clause_level_breakdown,clause_vectors,clause_polarity,categorisation
0,0,"(Translated by Google) Good hamburgers, delici...",5 days ago,4 stars,TFDB,"[{'hamburgers': ['delicious', 'pale'], 'ale': ...",[{'hamburgers': [[-0.65534 0.34034 0.30284...,"[{'hamburgers': [1.0, -0.21], 'ale': [0.8], 'c...","[{'hamburgers': 'Good hamburgers', 'ale': 'del...",[{'hamburgers': [[ 0.0068313 -0.15109 -0.012...,"[{'hamburgers': 0.7, 'ale': 0.395, 'cocktails'...","[{'hamburgers': 'food', 'ale': 'food', 'cockta..."


In [ ]:
def ml_df(df):
  newdf = pd.DataFrame()
  count = 0
  for r in range(0, len(df)):
      row = df.iloc[r, :]# Get entire row
      for feature in row["feature_descriptors"][0].keys():
        newdict = dict()
        for i in range(0, len(row["feature_descriptors"][0][feature])):
          
          # print(feature, row["feature_descriptors"][0][feature][i])
          newdict["feature"] = feature
          newdict["descriptor"] = row["feature_descriptors"][0][feature][i]
          # print(feature, pd.DataFrame(row["vector_descriptors"][0][feature][i].tolist()).T)
          try:
            tdf = pd.DataFrame(row["vector_descriptors"][0][feature][i].tolist()).T
          except:
            tdf = pd.DataFrame(np.zeros((1, 100)))
          for col in list(tdf.columns):
            # print(tdf.iloc[0, col] )
            newdict[str(col)] = str(tdf.iloc[0, col]) 
          # print(newdict)
          newdict["polarity"] = row["word_level_polarity"][0][feature][i]
          newrow = pd.DataFrame(newdict, index = [count])
          count+=1
          newdf = newdf.append(newrow)
  return newdf
ml_df(pdf)

,feature,descriptor,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,polarity
0,hamburgers,delicious,-0.6553400158882141,0.3403399884700775,0.302839994430542,-0.14854000508785248,0.176829993724823,0.3372499942779541,0.5125399827957153,0.04767699912190437,0.20363999903202057,-0.1697700023651123,0.06424400210380554,-0.030980000272393227,0.2926599979400635,0.2566800117492676,0.2662700116634369,0.5521000027656555,-0.19929000735282898,-0.4551199972629547,0.07585799694061279,0.6727499961853027,0.07455199956893921,0.21267999708652496,0.04304800182580948,-0.9397500157356262,0.16909000277519226,1.2609000205993652,-0.11849000304937363,0.19957999885082245,-0.7806699872016907,-0.9688000082969666,-0.2734900116920471,0.4715999960899353,-0.01145199965685606,-0.7421000003814697,0.4131700098514557,0.6046000123023987,-0.07598800212144852,0.21874000132083893,...,1.1601999998092651,-0.2936199903488159,-0.31742000579833984,0.2246199995279312,-1.1938999891281128,0.28202998638153076,-0.5876100063323975,-0.10937000066041946,-0.9409999847412109,-0.04688600078225136,0.3273699879646301,0.21783000230789185,0.5369799733161926,-0.20026999711990356,1.1719000339508057,-0.6695200204849243,-0.5335900187492371,0.40584999322891235,0.33660998940467834,-0.12291000038385391,-0.18885000050067902,-0.4521999955177307,0.6056100130081177,-0.46546998620033264,-0.4418100118637085,0.2503800094127655,0.1730400025844574,-0.5164700150489807,-0.2254599928855896,0.1645900011062622,0.27990999817848206,-0.42528998851776123,-0.46875,-1.1439000368118286,-0.6156799793243408,-0.42669999599456787,-0.6885300278663635,0.0895640030503273,0.7229999899864197,1.00
1,hamburgers,pale,-0.9040499925613403,-0.13759000599384308,0.11407999694347382,0.27199000120162964,0.2146500051021576,0.8402799963951111,0.4449999928474426,-0.10007999837398529,-0.02796499989926815,-0.032113999128341675,-0.8368399739265442,-0.058090999722480774,1.0611000061035156,0.6218600273132324,0.6469699740409851,0.10018999874591827,0.06756199896335602,-1.0015000104904175,0.807420015335083,-0.17146000266075134,-0.06947799772024155,0.4097999930381775,-0.2505199909210205,-0.12919999659061432,0.586329996585846,0.6360800266265869,0.761709988117218,-0.3003999888896942,-0.2205599993467331,-0.03821000084280968,0.8058300018310547,-1.1064000129699707,0.24055999517440796,-0.5697500109672546,0.5461900234222412,1.0729000568389893,1.1345000267028809,0.5817400217056274,...,0.7226499915122986,-1.214900016784668,0.33331000804901123,0.44117000699043274,-0.2973000109195709,0.5241000056266785,-0.23958000540733337,-0.7134199738502502,0.17768999934196472,0.4108000099658966,-0.9276000261306763,-0.26649001240730286,-0.40143999457359314,0.72257000207901,-0.23489999771118164,-0.1554899960756302,0.42026999592781067,0.13553999364376068,-0.1251000016927719,-0.3515999913215637,0.19262999296188354,0.10107000172138214,0.42364001274108887,-0.5255900025367737,-0.8121500015258789,0.6951799988746643,0.5623599886894226,0.8144000172615051,1.1359000205993652,0.5055800080299377,-0.2709600031375885,-0.450080007314682,0.5054000020027161,-0.6381099820137024,0.10409999638795853,-1.3976999521255493,0.4573900103569031,0.003285699989646673,0.3441300094127655,-0.21
2,ale,great,-0.013786000199615955,0.3821600079536438,0.5323600172996521,0.15261000394821167,-0.29693999886512756,-0.2055799961090088,-0.41846001148223877,-0.5843700170516968,-0.7735499739646912,-0.8786600232124329,-0.3785800039768219,-0.18515999615192413,-0.12800000607967377,-0.20584000647068024,-0.2292499989271164,-0.4259899854660034,0.3725000023841858,0.2607699930667877,-1.070199966430664,0.6291599869728088,-0.09146899729967117,0.7034800052642822,-0.49729999899864197,-0.7769100069999695,0.6604499816894531,0.09465000033378601,-0.4489299952983856,0.01891699992120266,0.33145999908447266,-0.3502199947834015,-0.3578900098800659,0.030313000082

In [ ]:
tr = ml_df(train)
te = ml_df(test)
va = ml_df(validate)


AttributeError: ignored

In [ ]:
tr.loc[(tr['polarity'] >= 0.5), 'polarity'] = 1
tr.loc[(tr['polarity'] < 0.5), 'polarity'] = 0

te.loc[(te['polarity'] >= 0.5), 'polarity'] = 1
te.loc[(te['polarity'] < 0.5), 'polarity'] = 0

va.loc[(va['polarity'] >= 0.5), 'polarity'] = 1
va.loc[(va['polarity'] < 0.5), 'polarity'] = 0

In [ ]:
tr

,feature,descriptor,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,polarity
0,review,really,-0.05434900149703026,0.622979998588562,0.6135299801826477,-0.6771000027656555,-0.6033599972724915,-0.07230699807405472,-0.10162000358104706,0.10333999991416931,0.35561999678611755,-0.4506700038909912,0.6038100123405457,0.005651600193232298,0.031624000519514084,0.0005028899759054184,-0.336760014295578,-0.19764000177383423,-0.5206900238990784,0.6268100142478943,-0.34042999148368835,0.9207199811935425,0.2261199951171875,0.6652799844741821,-0.03358900174498558,-0.8868600130081177,0.1870100051164627,0.49803000688552856,-0.42601001262664795,-0.7914599776268005,0.10025999695062637,-0.3525800108909607,-0.4201500117778778,1.0126999616622925,0.3920300006866455,0.0005932400235906243,0.39677000045776367,-0.13875000178813934,-0.39100998640060425,0.20225000381469727,...,1.2247999906539917,-0.8595899939537048,-0.19393999874591827,0.8626099824905396,0.27213001251220703,0.7953000068664551,0.19684000313282013,-0.28578001260757446,-0.44398000836372375,0.6618599891662598,-0.4915899932384491,-0.06710100173950195,0.021476000547409058,0.3518100082874298,0.269569993019104,0.13165999948978424,0.01895100064575672,-0.43446001410484314,0.1934799998998642,0.07383699715137482,-0.053676001727581024,-0.9460200071334839,-0.16189999878406525,-1.948799967765808,-0.5588600039482117,-0.2778699994087219,-0.2526000142097473,-0.6635199785232544,-0.6114199757575989,0.08040600270032883,-0.11174999922513962,-0.017304999753832817,-0.32350000739097595,-0.192440003156662,0.21202999353408813,-0.2917400002479553,-0.6142399907112122,-0.1594099998474121,1.0990999937057495,0.0
1,review,appreciate,0.02723800018429756,0.6368200182914734,0.18389999866485596,0.39653000235557556,-1.020799994468689,-0.022461000829935074,-0.9099100232124329,-0.24428999423980713,0.38971999287605286,-0.40494000911712646,-0.10864999890327454,-0.021376000717282295,-0.1722699999809265,-0.8767799735069275,-0.6312100291252136,-0.4646799862384796,-0.24075999855995178,0.5135300159454346,-0.6060799956321716,0.6947199702262878,-0.32117000222206116,0.11114999651908875,-0.610480010509491,-0.45590999722480774,0.0439240001142025,0.12014999985694885,-0.014162999577820301,-0.5374199748039246,0.7755699753761292,-0.5447900295257568,-0.5064600110054016,0.7690100073814392,0.04174499958753586,0.01954999938607216,0.3764599859714508,0.2078700065612793,0.020214000716805458,0.2575800120830536,...,1.152899980545044,-0.6842700242996216,-0.09283500164747238,-0.16575999557971954,0.03940799832344055,0.29330000281333923,0.3759300112724304,0.1028899997472763,-0.261680006980896,0.4323900043964386,-0.23337000608444214,0.1430400013923645,-0.5173599720001221,0.1806900054216385,0.3316200077533722,-0.25788000226020813,0.016390999779105186,0.1216999962925911,-0.021062999963760376,-0.11590000241994858,0.20091000199317932,-0.04730299860239029,-0.40634000301361084,-0.652209997177124,0.056605998426675797,-0.49022001028060913,0.09668999910354614,-0.8145800232887268,-0.5570099949836731,0.3064500093460083,0.1426199972629547,-0.3183700144290924,-0.686739981174469,-0.7022200226783752,-0.1805800050497055,-0.4521600008010864,-0.9611300230026245,-0.08127900213003159,0.8438599705696106,0.0
2,review,now,-0.01449500024318695,0.5910699963569641,0.7046899795532227,-0.40428999066352844,0.18782000243663788,0.12693999707698822,-0.14757999777793884,0.3458699882030487,0.22102999687194824,-0.28488001227378845,0.3504599928855896,0.06413699686527252,0.5915399789810181,0.11653999984264374,0.03990599885582924,-0.8012099862098694,0.37540000677108765,0.2934400141239166,-0.1036899983882904,0.2790699899196625,0.19986000657081604,0.6836900115013123,0.37553998827934265,-0.007272500079125166,0.047113001346588135,-0.25262999534606934,0.008200500160455704,-0.6978099942207336,-0.006852199789136648,-0.2143500000

In [ ]:
xtrain = tr.iloc[:,2:-1]
ytrain = tr.iloc[:, -1]

xtest = te.iloc[:,2:-1]
ytest = te.iloc[:, -1]

# xtrain = tr.iloc[:,2:-1]
# ytrain = tr.iloc[:, -1]

In [ ]:
# Load libraries
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

ada = AdaBoostClassifier(n_estimators=100)

In [ ]:
model = ada.fit(xtrain, ytrain)

In [ ]:
y_pred = model.predict(xtest)
print("Accuracy:",metrics.accuracy_score(ytest, y_pred))

Accuracy: 0.9510426110607434


In [ ]:
tr.groupby('polarity').count()

,feature,descriptor,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
polarity,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,...,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216,8216
1.0,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,...,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898,1898
